In [1]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime as dt
import script as sc
import re
import requests
from bs4 import BeautifulSoup
import gender_guesser.detector as gender
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.svm as svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

d = gender.Detector()


In [9]:
train = pd.read_csv("/Users/gusmoir/Desktop/Projects/SeniorProject/finalHelper.csv",encoding = "ISO-8859-1")
train = train.set_index('Symbol')
trainAnswers = train[['DaysBetterThanSP','daysProfit','daysProfitGrouped','Profitable','Safe','HomeRunDay','HomeRun']]
trainAnswers = trainAnswers[~trainAnswers.index.duplicated(keep='first')]
data = train
createCategories = ['Year','Month','Day','dayOfWeek','Sector','Industry','CEOGender','PresidentGender'
                ,'City','stateCountry','employeesGrouped','USACompany','FiscalMonth','FoundingDateGrouped'
                ,'usableCEOAge','usableCEOGender','usablePresidentAge','usablePresidentGender','yearDifferenceGrouped']
res1 = []
for index,row in data.iterrows():
    res1.append(index)
res = pd.DataFrame(res1,columns = ['Symbol'])
res = res.set_index('Symbol')
appl = []
for index, row in data.iterrows():
    appl.append(index)
for var in createCategories:
    temp = pd.get_dummies(data[var],prefix= var)
    temp['Symbol'] = appl
    temp2 = temp.set_index('Symbol')
    res = res.join(temp2)
    res = res[~res.index.duplicated(keep='first')]
data = data.drop(['DaysBetterThanSP','daysProfit','daysProfitGrouped','Profitable','Safe','HomeRunDay','HomeRun'],axis =1)

/Users/gusmoir/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1342,1425,1432,1543,1546,1549,1551,1552,1553,1562,1587,1588,1605,1608,1615,1619,1620,1621,1622,1629,1630,1632,1633,1640,1641,1642,1643,1644,1646) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
deciders = ['Safe','Profitable','HomeRun']

# Random Forrest test 

In [15]:
for response in deciders:
    highest = 0
    best = -1
    for i in range(1,50):
        rf = RandomForestClassifier(max_depth = i, random_state = 0)
        scores = cross_val_score(rf, res, trainAnswers[response], cv=5)
        if scores.mean() > highest:
            best = i
            highest = scores.mean()
    print('The best number of branches for Random Forrest for',response,'is',best,'with an accuracy of',highest)

The best number of branches for Random Forrest for Safe is 38 with an accuracy of 0.8421174302778178
The best number of branches for Random Forrest for Profitable is 39 with an accuracy of 0.6274644333101709
The best number of branches for Random Forrest for HomeRun is 39 with an accuracy of 0.9305232981936922


# K Nearest Neighbors test

In [17]:
for response in deciders:
    highest = 0
    best = -1
    for i in range(1,50):
        neigh = KNeighborsClassifier(n_neighbors = i)
        scores = cross_val_score(neigh, res, trainAnswers[response], cv=5)
        if scores.mean() > highest:
            highest = scores.mean()
            best = i
    print('The best number of neighbors for KNN for',response,'is',best,'with an accuracy of',highest)

The best number of neighbors for KNN for Safe is 16 with an accuracy of 0.8407671731380869
The best number of neighbors for KNN for Profitable is 26 with an accuracy of 0.6212456570237356
The best number of neighbors for KNN for HomeRun is 4 with an accuracy of 0.9305218372706443
